In [1]:
import os
import fnmatch

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from lr_video_face.plots import *

from lir import Xy_to_Xn, metrics

In [2]:

home = os.path.expanduser("~")

# SQL database path and name
input_dir = os.path.join(home, 'video_resources', 'sql_database')
#db_name = 'colab_main_dataset'
# Where results are saved
output_dir = os.path.join(home, 'video_resources', 'exp_output_SCFACELFW_nofiltros')

#Traducción de nombres
dic_emb = {'Facenet512': 'FaceNet', 'QMagFace':'QMagFace', 'ArcFace':'ArcFace'}
dic_qua = {'tface' : 'SDD-FIQA','ser_fiq': 'SER-FIQ'}

In [3]:

files = []
for root, dir, filenames in os.walk(output_dir):
    for filename in fnmatch.filter(filenames, 'dat*.pd'):
        files.append(os.path.join(root, filename))
   

In [4]:
def generate_plot(file):

    df0 = pd.read_pickle(file)

    if len(pd.unique(df0.Detector)) == 1 and len(pd.unique(df0.Calibrator)) == 1:

        cols = list(pd.unique(df0['Embedding Model']))
        rows = list(pd.unique(df0['Quality Model']))

        #generamos la gráfica con subplots
        fig,ax = plt.subplots(nrows = len(rows), ncols = len(cols), squeeze= False, figsize= (25,10))
        
        for index, df1 in df0.iterrows():

            col = cols.index(df1['Embedding Model'])
            row = rows.index(df1['Quality Model'])

            ax1 = ax[row][col]
            #ax1.ylabel = df1['Embedding Model']
            #ax1.set_title ( f"Quality Model: {dic_qua [df1['Quality Model']]}")
            ax1.set_title ( f"Embedding Model: {dic_emb [df1['Embedding Model']]}",fontsize = 14)

            subplot_new(ax1,df1.Results, df1.Cllr)
            #ax1.set(ylabel= f"Embedding Model: {dic_emb[df1['Embedding Model']]}\n Cllr")
            
            #ax1.set(ylabel= f"Quality Model: {dic_qua[df1['Quality Model']]}\n Cllr")
            ax1.set_ylabel(f"Quality Model: {dic_qua[df1['Quality Model']]}\n Cllr",fontsize = 14)
            ax1.set_xlabel('% of discarded pairs',fontsize=14)
        plt.suptitle('$C_{llr}$ Values',fontsize=18)

        #leyenda global
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14, borderaxespad=0.)
        fig.tight_layout()
        #savefig = os.path.join(self.experiments.output_dir, f"cllr_summary_ESX{self.experiments.embedding_model_as_scorer}")
        savefig = os.path.join(os.path.dirname(file), os.path.splitext(file)[0]+'_v4')
        plt.savefig(savefig, dpi=600)
        plt.close() 

In [5]:
def generate_latex(file):
    df0 = pd.read_pickle(file)


    
    

    if len(pd.unique(df0.Detector)) == 1 and len(pd.unique(df0.Calibrator)) == 1:
        df_latex = pd.DataFrame()

        for index, df1 in df0.iterrows():    

            emb_model = df1['Embedding Model']
            qua_model = df1['Quality Model']
            # the results are only received for 2015
            lrs_predicted = df1.Results["lrs_predicted"]
            y_test = df1.Results["y_test"]
            dropouts = df1.Results["quality_drops"]
            common_attributes = df1.Results['common_attributes']
    
            cllr_participants = np.mean(df1.Cllr)

            # imagen promedio (17 comparisons)
            lr_avg = df1.Results["lrs_predicted_2015"]
            y_avg = df1.Results["y_test_2015"]    
    
            cllr_avg = metrics.cllr(np.asarray(lr_avg), np.asarray(y_avg)) 
            df_latex = df_latex.append({'Embedding_model': 'Participants',
                                            'Quality_model': dic_qua[qua_model],
                                             'Cllr': cllr_participants},ignore_index = True)
            df_latex = df_latex.append({'Embedding_model': 'Average Image',
                                            'Quality_model': dic_qua[qua_model],
                                             'Cllr': cllr_avg},ignore_index = True)

            #get cllr per dropout

            
            for d in set(dropouts):
                lr_d = [lr for lr,dropout in zip(lrs_predicted,dropouts) if dropout== d ]
                y_d = [y for y,dropout in zip(y_test,dropouts) if dropout== d ]
                cllr_d = metrics.cllr(np.asarray(lr_d), np.asarray(y_d))

            #solo cambio en el momento de plotear
                df_latex = df_latex.append({'Embedding_model': dic_emb[emb_model],
                                            'Quality_model': dic_qua[qua_model],
                                             'Cllr': cllr_d,
                                             'Dropout': 100*(1-d)},ignore_index = True)

            df_latex.sort_values(by= 'Dropout', inplace = True)
            #df_latex.dropna(inplace= True)

            

            # get cllr per common attributes
            # filter results when dropout = 1 
            lrs = [lr for lr,drop in zip(lrs_predicted,dropouts) if drop== 1] 
            ys = [y for y,drop in zip(y_test,dropouts) if drop== 1]
            comatt = [ca for ca,drop in zip(common_attributes,dropouts) if drop== 1]


            #df_plot2 = pd.DataFrame()
            for c in set(comatt):
                lr_c = [lr for lr,com in zip(lrs,comatt) if com== c ]
                y_c = [y for y,comm in zip(ys,comatt) if comm== c ]
                cllr_c = metrics.cllr(np.asarray(lr_c), np.asarray(y_c))
                df_latex = df_latex.append({'Embedding_model': dic_emb[emb_model],
                                            'Quality_model': dic_qua[qua_model],
                                             'Cllr': cllr_c,
                                             'Common Attributes': c},ignore_index = True)
        
            df_latex.sort_values(by= 'Common Attributes', inplace = True)
            #df_latex.dropna(inplace= True)   

#----
        latex_table = df_latex.to_latex(index=False)
        savelatex = os.path.join(os.path.dirname(file), os.path.splitext(file)[0]+'.tex')

        with open (savelatex,'w') as f:
            f.write(latex_table)

In [6]:
for file in files:
    generate_plot(file)
    generate_latex(file)


0.26502477535199725 [0.00680578 0.03983876 0.1927867 ]
0.38368725686815675 [0.02250314 0.10462758 0.39442868]
0.3302946470911587 [0.01398214 0.06929824 0.28151047]
0.6095882801158683 [0.03798231 0.18222317 0.71804384]
0.4738528271056499 [0.02961343 0.14123964 0.54006069]
0.7694256350783979 [0.04995906 0.24020908 1.05156366]
0.9009018961888711 [0.06195274 0.30788105 1.38786554]
1.0536123919704516 [0.0729658  0.37699879 1.78681353]
1.1743067567673182 [0.0807548  0.44564926 2.16498204]
1.2856457012815143 [0.0878472  0.50714902 2.5593563 ]


/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The fram

0.5797857146838203 [0.02226147 0.12224422 0.56030613]
0.7953431542661418 [0.03574013 0.21082767 0.95285082]
0.6666193747183059 [0.02869907 0.16008607 0.68395645]
0.9994188771794252 [0.05440822 0.29448497 1.55055744]
0.9021805283092434 [0.0471377  0.24902976 1.23801982]
1.0910849548269337 [0.06472438 0.34273048 1.83821823]
1.1285958509606797 [0.06842147 0.36720815 1.97906736]
1.1839048060955024 [0.07702562 0.42129818 2.22781635]
1.2339000361279615 [0.08307525 0.46473607 2.42325749]
1.2856457012815141 [0.0878472  0.50714902 2.5593563 ]


/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The fram

0.7368571550482985 [0.35004734 0.63916733 1.14058228]
0.8896563439175895 [0.41118484 0.79576382 1.49770916]
0.826893025442619 [0.39615686 0.71838618 1.33421818]
0.9976616103813056 [0.41430144 0.96706115 1.7843199 ]
0.9388153514613831 [0.41212999 0.87558611 1.66545219]
1.0728161157812515 [0.42353124 1.10122787 1.87614757]
1.1979367327518293 [0.43193778 1.22636236 1.92590194]
1.2488633904283457 [0.43452444 1.2998731  1.96488134]
1.278126659314499 [0.43202519 1.35310101 1.99588725]
1.3080919694408588 [0.43183452 1.40121136 2.02476758]


...

/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The fram

0.6984107519567228 [0.0964662  0.29372839 0.85821133]
0.9260525172950091 [0.17444697 0.50504853 1.24385324]
0.8109596208345973 [0.1321851  0.41350719 1.07277387]
1.1352452559039288 [0.22740579 0.66845908 1.65945518]
1.033012877737916 [0.20865738 0.59774378 1.44199796]
1.2453896470963142 [0.23845806 0.73650724 1.89234399]
1.3682711142339552 [0.24070248 0.80610966 2.17401002]
1.4670190590984136 [0.24743263 0.88896455 2.4905092 ]
1.5434235925831357 [0.25464611 0.96979338 2.73871634]
1.6189093898020033 [0.25962607 1.06055178 2.97646382]


/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The fram

1.0752396350756417 [0.15606143 0.49191187 1.52441684]
1.2274191604539457 [0.18928717 0.63372269 1.85252077]
1.148028224779213 [0.17324855 0.53139634 1.62941243]
1.432415827256234 [0.2235759  0.8421598  2.31885204]
1.3454274878723935 [0.20435329 0.74265428 2.08097844]
1.480817595150996 [0.23555044 0.92109564 2.51704195]
1.5125264039111248 [0.23815231 0.9416457  2.65354265]
1.564576737326851 [0.24461388 0.99219625 2.7984037 ]
1.601500855319674 [0.25354387 1.04668568 2.92342332]
1.6189093898020033 [0.25962607 1.06055178 2.97646382]


/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_plot1 = df_plot1.append({'dropout': round(100*(1-d)), 'Cllr': cllr_d1,'Cllr2': cllr_d2},ignore_index = True)
/home/jmacarulla/lr-video-face/lr_video_face/plots.py:432: FutureWarning: The fram